In [1]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs | tail -n 1
!pip install -U xgboost==0.90 | tail -n 1

In [4]:
from ibm_watson_machine_learning.helpers import DataConnection, S3Connection, S3Location

training_data_reference = [DataConnection(
    connection=S3Connection(
        api_key='yIFSGj3F7qcScFQqdVDsA0Fr-nzrUcwOyL_Lj5WE-FDs',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3.eu-geo.objectstorage.softlayer.net'
    ),
        location=S3Location(
        bucket='diabetesprediction-donotdelete-pr-erhquxz5qwghet',
        path='diabetes.csv'
    )),
]
training_result_reference = DataConnection(
    connection=S3Connection(
        api_key='yIFSGj3F7qcScFQqdVDsA0Fr-nzrUcwOyL_Lj5WE-FDs',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3.eu-geo.objectstorage.softlayer.net'
    ),
    location=S3Location(
        bucket='diabetesprediction-donotdelete-pr-erhquxz5qwghet',
        path='auto_ml/eaa80418-a9ba-4ba4-8469-029ca938c6df/wml_data/b55442a1-2c2b-4a54-a2d5-4ea66326059c/data/automl',
        model_location='auto_ml/eaa80418-a9ba-4ba4-8469-029ca938c6df/wml_data/b55442a1-2c2b-4a54-a2d5-4ea66326059c/data/automl/hpo_c_output/Pipeline1/model.pickle',
        training_status='auto_ml/eaa80418-a9ba-4ba4-8469-029ca938c6df/wml_data/b55442a1-2c2b-4a54-a2d5-4ea66326059c/training-status.json'
    ))

In [5]:
experiment_metadata = dict(
   prediction_type='classification',
   prediction_column='Outcome',
   test_size=0.1,
   scoring='accuracy',
   project_id='fe7c7525-883f-49c5-8034-4476e03e0641',
   space_id='None',
   deployment_url='https://eu-gb.ml.cloud.ibm.com',
   csv_separator=',',
   random_state=33,
   max_number_of_estimators=2,
   daub_include_only_estimators=None,
   training_data_reference=training_data_reference,
   training_result_reference=training_result_reference,
   positive_label=1
)

In [6]:
df = training_data_reference[0].read(csv_separator=experiment_metadata['csv_separator'])
df.dropna('rows', how='any', subset=[experiment_metadata['prediction_column']], inplace=True)

<a id="split"></a>
##  Train and test data split

In [7]:
from sklearn.model_selection import train_test_split

df.drop_duplicates(inplace=True)
X = df.drop([experiment_metadata['prediction_column']], axis=1).values
y = df[experiment_metadata['prediction_column']].values

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=experiment_metadata['test_size'],
                                                    stratify=y, random_state=experiment_metadata['random_state'])

Import statements.

In [8]:
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TNoOp
from xgboost import XGBClassifier

#### Pre-processing & Estimator.

In [9]:
numpy_column_selector_0 = NumpyColumnSelector(columns=[0, 2, 3, 7])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "float_int_num",
        "float_int_num",
        "float_int_num",
        "float_int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=100001
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=100001,
    filling_values_list=[100001, 100001, 100001, 100001],
    known_values_list=[
        [
            0.0,
            1.0,
            2.0,
            3.0,
            4.0,
            5.0,
            6.0,
            7.0,
            8.0,
            9.0,
            10.0,
            11.0,
            12.0,
            13.0,
            14.0,
            15.0,
            17.0,
        ],
        [
            0.0,
            24.0,
            30.0,
            38.0,
            40.0,
            44.0,
            46.0,
            48.0,
            50.0,
            52.0,
            54.0,
            55.0,
            56.0,
            58.0,
            60.0,
            61.0,
            62.0,
            64.0,
            65.0,
            66.0,
            68.0,
            70.0,
            72.0,
            74.0,
            75.0,
            76.0,
            78.0,
            80.0,
            82.0,
            84.0,
            85.0,
            86.0,
            88.0,
            90.0,
            92.0,
            94.0,
            95.0,
            96.0,
            98.0,
            100.0,
            104.0,
            106.0,
            108.0,
            110.0,
            114.0,
            122.0,
        ],
        [
            0.0,
            7.0,
            8.0,
            10.0,
            11.0,
            12.0,
            13.0,
            14.0,
            15.0,
            16.0,
            17.0,
            18.0,
            19.0,
            20.0,
            21.0,
            22.0,
            23.0,
            24.0,
            25.0,
            26.0,
            27.0,
            28.0,
            29.0,
            30.0,
            31.0,
            32.0,
            33.0,
            34.0,
            35.0,
            36.0,
            37.0,
            38.0,
            39.0,
            40.0,
            41.0,
            42.0,
            43.0,
            44.0,
            45.0,
            46.0,
            47.0,
            48.0,
            49.0,
            50.0,
            52.0,
            54.0,
            56.0,
            59.0,
            60.0,
            63.0,
            99.0,
            110.0,
        ],
        [
            21.0,
            22.0,
            23.0,
            24.0,
            25.0,
            26.0,
            27.0,
            28.0,
            29.0,
            30.0,
            31.0,
            32.0,
            33.0,
            34.0,
            35.0,
            36.0,
            37.0,
            38.0,
            39.0,
            40.0,
            41.0,
            42.0,
            43.0,
            44.0,
            45.0,
            46.0,
            47.0,
            48.0,
            49.0,
            50.0,
            51.0,
            52.0,
            53.0,
            54.0,
            55.0,
            56.0,
            57.0,
            58.0,
            59.0,
            60.0,
            61.0,
            62.0,
            63.0,
            65.0,
            66.0,
            67.0,
            68.0,
            69.0,
            70.0,
            72.0,
            81.0,
        ],
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=100001,
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[1, 4, 5, 6])
float_str2_float = FloatStr2Float(
    dtypes_list=["float_int_num", "float_int_num", "float_num", "float_num"],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0, permutation_indices=[0, 2, 3, 7, 1, 4, 5, 6]
)
t_no_op = TNoOp(
    fun=None, name="no_action", datatypes="x", feat_constraints=[]
)
xgb_classifier = XGBClassifier(
    gamma=1.0802271427722739e-06,
    learning_rate=0.02000116894105963,
    max_depth=1,
    min_child_weight=2,
    n_estimators=197,
    random_state=33,
    reg_alpha=0.9999960244176358,
    reg_lambda=0.9999995122571778,
    scale_pos_weight=1.3930942044422527,
    silent=True,
    subsample=0.9999997991531601,
    verbosity=0,
    tree_method="hist",
)


In [10]:
pipeline = make_pipeline(union, numpy_permute_array, t_no_op, xgb_classifier)

In [11]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

In [12]:
pipeline.fit(train_X,train_y)

Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('pipeline-1',
                                                 Pipeline(memory=None,
                                                          steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               2,
                                                                                               3,
                                                                                               7])),
                                                                 ('compressstrings',
                                                                  CompressStrings(activate_flag=True,
                                                                                  compress_type='

In [13]:
score = scorer(pipeline, test_X, test_y)
print(score)

0.7866666666666666
